# Export EDF

This notebook demonstrates the process of exporting DiveDB data as an EDF file.

While under development, it also contains the prototype (non-library) code; that'll be deleted when this notebook is ready to be merged into the main branch.

Punch list:
- [x] Make a list
- [ ] Understand task :) 
- [ ] Prototype:
    - [ ] Load metadata
    - [ ] Load signals
    - [ ] Generate EDF file 
        - [ ] Can mne serve our needs here? Check if multiple sample rates, arbitrary metadata
        - [ ] Decide if different library OR extend mne 
    - [ ] Test EDF file can be opened externally (e.g. via EDF.jl)
- [ ] In tests, write (failing) test for basic new functionality
- [ ] Turn prototype into library code
- [ ] Write up edge case tests
    - [ ] Make 'em pass OR file 'em
- [ ] Clean up this notebook (delete this punch list!)
- [ ] Mark PR ready for review

### Prototype

In [ ]:
# 1. Get metadata
